<a href="https://colab.research.google.com/github/GiX7000/10-machine-learning-algorithms-from-scratch/blob/main/05_Decision_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementation of a Decision Tree classifier

In this notebook, a quick and simple implementation of a [Decision Tree](https://www.youtube.com/watch?v=JcI5E2Ng6r4) algorithm is presented.

In [ ]:
# imports
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
import matplotlib.pyplot as plt
from collections import Counter

## 1. Download [breast cancer dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html) and split it to train and test sets.

In [ ]:
# get the dataset
data = datasets.load_breast_cancer()
X, y = data.data, data.target

# split iiit to train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# let's see some things about the data
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print(X_train.dtype, X_test.dtype)
print(X_train[5])
print(y_train[5])
print(np.unique(y_train))

(455, 30) (455,) (114, 30) (114,)
float64 float64
[1.625e+01 1.951e+01 1.098e+02 8.158e+02 1.026e-01 1.893e-01 2.236e-01
 9.194e-02 2.151e-01 6.578e-02 3.147e-01 9.857e-01 3.070e+00 3.312e+01
 9.197e-03 5.470e-02 8.079e-02 2.215e-02 2.773e-02 6.355e-03 1.739e+01
 2.305e+01 1.221e+02 9.397e+02 1.377e-01 4.462e-01 5.897e-01 1.775e-01
 3.318e-01 9.136e-02]
0
[0 1]


## 2. Create a custom Decision Tree model.

A tree is comprised of Nodes, so we create two classes here. All the steps below are presented step by step [here](https://www.youtube.com/watch?v=NxEHSAfFlK8).

In [ ]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None

In [ ]:
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None

    # fit function for training
    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)

    # helper recursive function to create the tree
    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # 1. check the stopping criteria
        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        # 2. find the best split
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # 3. create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)


    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # calculate the information gain
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold


    def _information_gain(self, y, X_column, threshold):
        # parent entropy
        parent_entropy = self._entropy(y)

        # create children
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # calculate the weighted avg. entropy of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p>0])


    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

## 3. Train, predict and evaluate the model.

In [ ]:
# create ana instance of our classifier
clf_decision_trees = DecisionTree()

# train the classifier
clf_decision_trees.fit(X_train, y_train)

In [ ]:
# predict on test set
predictions = clf_decision_trees.predict(X_test)

# Print the predictions and the actual labels
print("Predictions:", np.array(predictions))
print("Actual Labels:", y_test)

Predictions: [1 1 1 1 1 1 0 1 0 0 0 1 1 0 1 0 1 1 1 0 1 0 0 0 0 1 0 1 1 1 1 1 0 1 1 1 1
 0 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 0 1 0
 1 1 1 0 1 0 1 1 1 1 0 0 0 0 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0
 1 0 0]
Actual Labels: [1 1 1 1 1 1 0 1 0 0 0 1 1 1 1 0 1 1 1 0 1 0 0 0 0 1 0 1 1 1 1 1 0 1 1 1 1
 0 0 1 0 1 0 1 1 1 1 0 0 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 1 0 1 0
 1 1 1 0 1 0 1 1 1 0 0 0 0 0 1 0 1 0 1 0 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0
 1 0 0]


Let's use how good our model is.

In [ ]:
# accuracy function
def accuracy(y_true, y_pred):
  return np.sum(y_true == y_pred) / len(y_true)

print(accuracy(y_test, predictions))

0.9385964912280702


Accuracy is almost 94%, but what happens if we change the max_depth or the min_samples_split parameters?

Let's compare it now to the one of scikit-learn library.

## 4. Compare with the Decision Tree classifier from scikit-learn library.

Let's see now, what results a [Decision Trees classifier](https://scikit-learn.org/1.5/modules/tree.html) from scikit-learn library gives us.

In [ ]:
# let's compare now with the accuracy that sklearn gives us
from sklearn import tree

clf_sklearn = tree.DecisionTreeClassifier()
clf_sklearn.fit(X_train, y_train)
sklearn_predictions = clf_sklearn.predict(X_test)

In [ ]:
# let's check the accuracy now
print(accuracy(y_test, sklearn_predictions))

0.8947368421052632


Our custom decision tree classifier got a better accuracy than sklearn's one here.